In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [54]:
#read new combined cleaned dataset
df = pd.read_csv("wsb.csv", index_col=0)



In [104]:
#sort by num_mentions rolled up into ticker
df_num_mentions =df.sort_values(["num_mentions","ticker"],ascending=False).groupby("ticker")
df_num_mentions = df.groupby('ticker')['num_mentions'].sum().sort_values(ascending=False)
df_num_mentions.head(20)

#df_num_mentions.to_csv('test.csv')

ticker
GME     122643.0
BB       41669.0
AMC      38061.0
PLTR     20785.0
CLOV     15889.0
WISH     10965.0
SPY      10823.0
RKT      10520.0
TSLA     10194.0
CLNE      8252.0
NOK       8133.0
UWMC      6239.0
AMD       5620.0
TLRY      5362.0
SNDL      5133.0
AAPL      5040.0
WKHS      4565.0
MVIS      4465.0
NIO       4198.0
ALL       4063.0
Name: num_mentions, dtype: float64

In [113]:
#sort by pct_mentions rolled up into ticker
df_pct_mentions = df.sort_values(["pct_mentions","ticker"],ascending=False).groupby("ticker")
df_pct_mentions = df.groupby('ticker')['pct_mentions'].mean().sort_values(ascending=False)
df_pct_mentions.head(20)

#df_pct_mentions.to_csv('test.csv')

ticker
GME     16.591133
AMC      5.882051
NAPA     5.666667
BB       4.985222
CLOV     4.517857
PLTR     3.960396
SPY      3.763682
IRNT     3.555556
WISH     2.788194
SDC      2.658730
DOLE     2.500000
TSLA     2.217822
HOOD     2.147727
OPAD     2.000000
MVIS     1.891608
TMC      1.861111
MNMD     1.756410
CLNE     1.679688
RKT      1.640541
MVST     1.500000
Name: pct_mentions, dtype: float64

In [105]:
#sort by pos_count rolled up into ticker
df_pos_count =df.sort_values(["pos_count","ticker"],ascending=False).groupby("ticker")
df_pos_count = df.groupby('ticker')['pos_count'].sum().sort_values(ascending=False)
df_pos_count.head(20)

#df_pos_count.to_csv('test.csv')

ticker
GME     57609.0
BB      17018.0
AMC     16999.0
PLTR     8497.0
WISH     7928.0
CLOV     5964.0
RKT      5233.0
TSLA     4434.0
SPY      4033.0
CLNE     3651.0
UWMC     3128.0
NOK      3075.0
AMD      2974.0
TLRY     2351.0
MVIS     1995.0
WKHS     1968.0
APHA     1941.0
SNDL     1859.0
CLF      1763.0
NIO      1643.0
Name: pos_count, dtype: float64

In [106]:
#sort by neg_count rolled up into ticker
df_neg_count =df.sort_values(["neg_count","ticker"],ascending=False).groupby("ticker")
df_neg_count = df.groupby('ticker')['neg_count'].sum().sort_values(ascending=False)
df_neg_count.head(20)

#df_neg_count.to_csv('test.csv')

ticker
GME     32026.0
AMC      8611.0
BB       7202.0
PLTR     4758.0
SPY      3128.0
CLOV     2654.0
TSLA     2554.0
RKT      2518.0
ALL      1552.0
NOK      1478.0
NIO      1270.0
CLNE     1242.0
UWMC     1211.0
AMD      1189.0
WISH     1185.0
AAPL      976.0
SNDL      967.0
BABA      934.0
MVIS      906.0
TLRY      840.0
Name: neg_count, dtype: float64